### start

In [1]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt
import sklearn

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

# import local modules
# pyright: reportMissingImports=false
sys.path.append('../src')

import sage_utils.config_validation as ucv
import sage_utils.s3_utils as s3u

import sage_wallet_insights.model_evaluation as sime

import sage_wallet_modeling.wallet_modeler as wm
from sage_wallet_modeling.wallet_preprocessor import SageWalletsPreprocessor
import sage_wallet_modeling.wallet_script_modeler as sm
import sage_wallet_modeling.workflow_orchestrator as wo

import script_modeling.entry_helpers as h
# import script_modeling.temporal_cv_model_entry as tme


# import data-science modules
sys.path.append(str(Path("..") / ".." / "data-science" / "src"))
import wallet_insights.model_evaluation as wime
import utils as u
from utils import ConfigError



# reload all modules
modules = [
    ucv, s3u,
    sime,
    wm, sm, wo,
    h,
    wime,
    u,
]

# import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
# ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#b45827"


# load all configs
sage_wallets_config = ucv.load_sage_wallets_config(Path('../config/sage_wallets_config.yaml'))
sage_wallets_modeling_config = ucv.load_sage_wallets_modeling_config(Path('../config/sage_wallets_modeling_config.yaml'))


u.export_code(
    code_directories=[
        'sage_wallet_insights',
        'sage_wallet_modeling',
        # 'sage_utils',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
    output_file="temp/sagemaker_code.py"
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/jeremymeadow/Library/Application Support/sagemaker/config.yaml


[01/Aug/25 16:53:35] INFO [utils.export_code:1659] Consolidation complete. All files are saved in temp/sagemaker_code.py
[01/Aug/25 16:53:35] MILESTONE [3929128312.<module>:134] Good morning, let's get to work


# Model Training

## Multi Model Sequence

### Initiate orchestrator

In [5]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)

### Load, preprocess, and upload all data

#### load data

In [13]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

workflow_orch.load_all_training_data(date_suffixes)

[01/Aug/25 14:16:42] MILESTONE [workflow_orchestrator.load_all_training_data:124] <DEV> Loading training data for 5 periods: ['231008', '231107', '240306', '241101', '250301']
[01/Aug/25 14:16:43] INFO [workflow_orchestrator.load_all_training_data:140] Training data loaded successfully: 40,000 total rows and 9 offsets for each date_suffix.


##### inspect features

In [ ]:
# Create combined NaN count and describe statistics
first_key = next(iter(workflow_orch.training_data))
nan_counts = workflow_orch.training_data[first_key]['x_train'].isna().sum()
describe_stats = workflow_orch.training_data[first_key]['x_train'].describe().T

# Combine into single DataFrame
combined_stats = pd.concat([
    nan_counts.rename('nan_count'),
    describe_stats
], axis=1).sort_index()

u.display_full(combined_stats.sort_index())

#### preprocess all data

In [14]:
workflow_orch.preprocess_all_training_data()

[01/Aug/25 14:16:44] INFO [workflow_orchestrator.preprocess_all_training_data:171] Preprocessing 5 date periods...
[01/Aug/25 14:16:44] INFO [wallet_preprocessor.preprocess_training_data:77] Starting preprocessing for SageMaker XGBoost compatibility...
[01/Aug/25 14:16:44] INFO [wallet_preprocessor._handle_missing_values:237] Filled NaN values in 100 columns for x_train.
[01/Aug/25 14:16:44] INFO [wallet_preprocessor.preprocess_y_data:276] Applied classification threshold 0.05 to y_train: 316 positive (31.6%), 684 negative (68.4%)
[01/Aug/25 14:16:44] INFO [wallet_preprocessor._combine_x_y_data:387] Merged y df with target var cw_crypto_net_gain/crypto_inflows/winsorized with X data.
[01/Aug/25 14:16:44] INFO [wallet_preprocessor._save_preprocessed_df:433] Saved preprocessed train split to ../s3_uploads/wallet_training_data_preprocessed/dda_920_eval_models_nginv_040_dev/231008/train.csv
[01/Aug/25 14:16:44] INFO [wallet_preprocessor.preprocess_training_data:119] Preprocessed train: 1,0

#### upload all data

In [16]:
# Upload and retrieve URIs
workflow_orch.upload_all_training_data(overwrite_existing=True)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[01/Aug/25 14:17:13] MILESTONE [workflow_orchestrator._confirm_upload:853] <DEV> Ready to upload 20000 total rows (33.8 MB) of preprocessed training data across 5 date folders.
[01/Aug/25 14:17:13] MILESTONE [workflow_orchestrator._confirm_upload:869] This upload will overwrite existing files.
[01/Aug/25 14:17:13] INFO [workflow_orchestrator._confirm_upload:873] Target: s3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/[DATE]/
[01/Aug/25 14:17:15] INFO [workflow_orchestrator.upload_all_training_data:218] Beginning approved upload...
[01/Aug/25 14:17:15] INFO [workflow_orchestrator.upload_all_training_data:223] Uploading data for 5 date periods with 4 threads...
[01/Aug/25 14:17:20] INFO [workflow_orchestrator._upload_csv_files:921] Uploaded train to s3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/240306/train.csv
[01/Aug/25 14:17:20] INFO [workflow_orchestrator._upload_csv_files:921] Uploaded train to s3://wallet-training-data/trainin

### Train all models

In [ ]:
workflow_orch.load_all_training_data(date_suffixes)
modeling_results = workflow_orch.train_all_models()

### Predict with all models

In [9]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[01/Aug/25 12:48:22] MILESTONE [workflow_orchestrator.load_all_training_data:121] <DEV> Loading training data for 4 periods: ['231008', '231107', '240306', '250301']
[01/Aug/25 12:48:22] INFO [workflow_orchestrator.load_all_training_data:137] Training data loaded successfully: 32,000 total rows and 9 offsets for each date_suffix.


In [ ]:
result = workflow_orch.predict_with_all_models()

### Evaluate all models

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)

evaluators = workflow_orch.evaluate_all_models()


In [ ]:
for e in evaluators:
    evaluators[e].plot_wallet_evaluation()

### DDA 923 temporal cv model

In [18]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '241101',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[01/Aug/25 15:52:50] MILESTONE [workflow_orchestrator.load_all_training_data:123] <DEV> Loading training data for 5 periods: ['231008', '231107', '240306', '241101', '250301']
[01/Aug/25 15:52:50] INFO [workflow_orchestrator.load_all_training_data:139] Training data loaded successfully: 40,000 total rows and 9 offsets for each date_suffix.


In [17]:
cv_s3_uri = workflow_orch.compile_temporal_cv_files(date_suffixes, s3_uris)
cv_s3_uri

[01/Aug/25 15:52:06] INFO [workflow_orchestrator.compile_temporal_cv_files:451] Uploaded 0 CV files to s3://wallet-training-data/training-data-temporal-cvs/dda-920-nginv-040


's3://wallet-training-data/training-data-temporal-cvs/dda-920-nginv-040'

In [19]:
sm.train_temporal_cv_script_model(sage_wallets_config,sage_wallets_modeling_config,cv_s3_uri)

[01/Aug/25 15:52:54] WARNING [wallet_script_modeler._prepare_hyperparameters:38] Ignoring unsupported hyperparameters: {'colsample_bytree'}
[01/Aug/25 15:52:54] INFO [image_uris._validate_py_version_and_set_if_needed:603] Ignoring unnecessary Python version: py3.
[01/Aug/25 15:52:54] INFO [image_uris._processor:530] Ignoring unnecessary instance type: ml.m5.large.
[01/Aug/25 15:52:54] INFO [wallet_script_modeler.train_temporal_cv_script_model:189] Launching script-mode CV training job: wallet-script-cv-dda-920-nginv-040-20250801-155254
[01/Aug/25 15:52:54] INFO [telemetry_logging.wrapper:92] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[01/Aug/25 15:52

2025-08-01 22:52:55 Starting - Starting the training job...
2025-08-01 22:53:10 Starting - Preparing the instances for training...
2025-08-01 22:53:33 Downloading - Downloading input data...
2025-08-01 22:54:18 Downloading - Downloading the training image......
2025-08-01 22:55:24 Training - Training image download completed. Training in progress../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-08-01 22:55:27.524 ip-10-0-216-117.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-08-01 22:55:27.561 ip-10-0-216-117.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-08-01:22:55:28:INFO] Imported framework sagem

[01/Aug/25 15:56:16] INFO [wallet_script_modeler.train_temporal_cv_script_model:193] Script-mode CV training completed. Model URI: s3://wallet-script-models/model-outputs/dda-920-nginv-040/cv/wallet-script-cv-dda-920-nginv-040-20250801-155254/wallet-script-cv-dda-920-nginv-040-20250801-155254/output/model.tar.gz


{'model_uri': 's3://wallet-script-models/model-outputs/dda-920-nginv-040/cv/wallet-script-cv-dda-920-nginv-040-20250801-155254/wallet-script-cv-dda-920-nginv-040-20250801-155254/output/model.tar.gz',
 'training_job_name': 'wallet-script-cv-dda-920-nginv-040-20250801-155254'}

## Single Model Sequence

### Train Single Model

#### retrieve file uris

In [6]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)



date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

[01/Aug/25 15:11:16] INFO [credentials.load:1352] Found credentials in shared credentials file: ~/.aws/credentials


{'231008': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231008/train.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231008/test.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231008/eval.csv',
  'val': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231008/val.csv'},
 '231107': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231107/train.csv',
  'test': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231107/test.csv',
  'eval': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231107/eval.csv',
  'val': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/231107/val.csv'},
 '240306': {'train': 's3://wallet-training-data/training-data-preprocessed/dda-920-nginv-040-dev/240306/train.csv',
  'test': 's3://wallet-trainin

#### train model

In [6]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffix = '231008'

modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)



In [7]:
modeling_results = modeler.train_model()

[01/Aug/25 13:03:09] WARNING [wallet_script_modeler.launch_script_mode_job:93] Ignoring unsupported hyperparameters for script-mode: {'colsample_bytree'}
[01/Aug/25 13:03:09] INFO [image_uris._validate_py_version_and_set_if_needed:603] Ignoring unnecessary Python version: py3.
[01/Aug/25 13:03:09] INFO [image_uris._processor:530] Ignoring unnecessary instance type: ml.m5.large.
[01/Aug/25 13:03:09] INFO [wallet_script_modeler.launch_script_mode_job:111] Launching script-mode training job: wallet-script-dda-920-nginv-040-231008-20250801-130309
[01/Aug/25 13:03:09] INFO [telemetry_logging.wrapper:92] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[01/Aug/2

2025-08-01 20:03:10 Starting - Starting the training job...
2025-08-01 20:03:30 Starting - Preparing the instances for training...
2025-08-01 20:03:52 Downloading - Downloading input data...
2025-08-01 20:04:37 Downloading - Downloading the training image......
2025-08-01 20:05:48 Training - Training image download completed. Training in progress.
2025-08-01 20:05:48 Uploading - Uploading generated training model./miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-08-01 20:05:42.958 ip-10-2-71-74.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-08-01 20:05:42.985 ip-10-2-71-74.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disable

[01/Aug/25 13:06:34] INFO [wallet_script_modeler.launch_script_mode_job:119] Script-mode training completed. Model URI: s3://wallet-script-models/model-outputs/dda-920-nginv-040/231008/wallet-script-dda-920-nginv-040-231008-20250801-130309/output/model.tar.gz


### Score validation set

#### get existing model uri

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))



date_suffix = '231008'

# Generate URIs for specific dates without any prior setup
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
s3_uris = workflow_orch.retrieve_training_data_uris([date_suffix])

# Launch modeler
modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)


model_metadata = modeler.load_existing_model()

#### predict with batch transform

In [ ]:
transform_results = modeler.predict_with_batch_transform('val')

In [ ]:
transform_results = modeler.predict_with_batch_transform('test')

### Predict Using Endpoints

#### deploy endpoint for predictions

In [ ]:
modeler.list_active_endpoints()

In [ ]:
modeler.deploy_endpoint()

#### load and preprocess test/val dfs

In [ ]:
x_test = pd.read_parquet(
    Path("../s3_uploads/wallet_training_data_queue") /
    sage_wallets_config['training_data']['local_directory'] /
    f"x_test_{date_suffix}.parquet"
)
x_val = pd.read_parquet(
    Path("../s3_uploads/wallet_training_data_queue") /
    sage_wallets_config['training_data']['local_directory'] /
    f"x_val_{date_suffix}.parquet"
)




In [ ]:
preprocessor = SageWalletsPreprocessor(sage_wallets_config)
x_test_processed = preprocessor.preprocess_x_df(x_test)
x_val_processed = preprocessor.preprocess_x_df(x_val)

logger.info(f"Preprocessed X_test {x_test_processed.shape} and "
            f"X_val {x_val_processed.shape}.")

#### predict

In [ ]:
y_test_pred = modeler.predict_using_endpoint(x_test_processed, 'test')
y_val_pred = modeler.predict_using_endpoint(x_val_processed, 'val')

#### delete all endpoints

In [ ]:
modeler.delete_all_endpoints()

In [ ]:
modeler.list_active_endpoints()
modeler.list_all_endpoints()

## Script model training

In [12]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]

date_suffix = '231008'


workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[01/Aug/25 15:40:31] MILESTONE [workflow_orchestrator.load_all_training_data:123] <DEV> Loading training data for 4 periods: ['231008', '231107', '240306', '250301']
[01/Aug/25 15:40:32] INFO [workflow_orchestrator.load_all_training_data:139] Training data loaded successfully: 32,000 total rows and 9 offsets for each date_suffix.


In [13]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffix = '231008'

modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)



In [14]:
sm.train_single_period_script_model(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)

[01/Aug/25 15:40:34] WARNING [wallet_script_modeler._prepare_hyperparameters:38] Ignoring unsupported hyperparameters: {'colsample_bytree'}
[01/Aug/25 15:40:34] INFO [image_uris._validate_py_version_and_set_if_needed:603] Ignoring unnecessary Python version: py3.
[01/Aug/25 15:40:34] INFO [image_uris._processor:530] Ignoring unnecessary instance type: ml.m5.large.
[01/Aug/25 15:40:34] INFO [wallet_script_modeler.train_single_period_script_model:121] Launching script-mode training job: wallet-script-dda-920-nginv-040-231008-20250801-154034
[01/Aug/25 15:40:34] INFO [telemetry_logging.wrapper:92] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[01/Aug/25 15

2025-08-01 22:40:36 Starting - Starting the training job...
2025-08-01 22:40:50 Starting - Preparing the instances for training...
2025-08-01 22:41:12 Downloading - Downloading input data...
2025-08-01 22:41:54 Downloading - Downloading the training image......
2025-08-01 22:43:11 Training - Training image download completed. Training in progress.
2025-08-01 22:43:11 Uploading - Uploading generated training model/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-08-01 22:43:02.619 ip-10-0-171-217.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-08-01 22:43:02.643 ip-10-0-171-217.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disa

[01/Aug/25 15:43:57] INFO [wallet_script_modeler.train_single_period_script_model:129] Script-mode training completed. Model URI: s3://wallet-script-models/model-outputs/dda-920-nginv-040/231008/wallet-script-dda-920-nginv-040-231008-20250801-154034/output/model.tar.gz


{'model_uri': 's3://wallet-script-models/model-outputs/dda-920-nginv-040/231008/wallet-script-dda-920-nginv-040-231008-20250801-154034/output/model.tar.gz',
 'training_job_name': 'wallet-script-dda-920-nginv-040-231008-20250801-154034',
 'date_suffix': '231008'}

In [13]:
from sagemaker.session import Session
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter

s3_data = {
    "train":      s3_uris[date_suffix]['train'],
    "validation": s3_uris[date_suffix]['val']
}
model_output_path = (f"s3://wallet-script-models/model-outputs")

estimator = XGBoost(
    entry_point         = "script_mode_modeler.py",     # path relative to notebook
    source_dir          = "../src/script_modeling/",    # folder that holds the script
    base_job_name       = "dda-922-v4",                 # uploaded and immutable script version
    framework_version   = sage_wallets_modeling_config['framework']['version'],
    role                = sage_wallets_config['aws']['modeler_arn'],
    instance_type       = sage_wallets_modeling_config['metaparams']['instance_type'],
    instance_count      = sage_wallets_modeling_config['metaparams']['instance_count'],
    output_path         = model_output_path,
    hyperparameters = {
        "eta": 0.1,
        "max_depth": 6,
        "subsample": 1.0,
        "num_boost_round": 50,            # small for a smoke-test
        "early_stopping_rounds": 10
    },
    # metric_definitions=[                  # regex must match the print line
    #     {"Name": "cv_auc_pr", "Regex": "validation:cv_auc_pr=(.*)"}
    # ],
)

# one-shot fit (no tuning yet)
estimator.fit(s3_data, logs="All")

[01/Aug/25 11:59:45] INFO [image_uris._validate_py_version_and_set_if_needed:603] Ignoring unnecessary Python version: py3.
[01/Aug/25 11:59:45] INFO [image_uris._processor:530] Ignoring unnecessary instance type: ml.m5.large.
[01/Aug/25 11:59:45] INFO [telemetry_logging.wrapper:92] SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
[01/Aug/25 11:59:45] INFO [session.submit:1053] Creating training-job with name: dda-922-v4-2025-08-01-18-59-45-362


2025-08-01 18:59:47 Starting - Starting the training job...
2025-08-01 19:00:01 Starting - Preparing the instances for training...
2025-08-01 19:00:23 Downloading - Downloading input data...
2025-08-01 19:01:09 Downloading - Downloading the training image......
2025-08-01 19:02:15 Training - Training image download completed. Training in progress.../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-08-01 19:02:17.367 ip-10-2-69-59.us-west-2.compute.internal:8 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-08-01 19:02:17.391 ip-10-2-69-59.us-west-2.compute.internal:8 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-08-01:19:02:17:INFO] Imported framework sagemake

# Model Evaluation

### Analyze predictions

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    # '240306'
]
date_suffix = date_suffixes[0]

In [ ]:
y_test_pred = sime.load_bt_sagemaker_predictions('test', sage_wallets_config, date_suffix)
y_val_pred = sime.load_bt_sagemaker_predictions('val', sage_wallets_config, date_suffix)

In [ ]:
# Single function call for complete evaluation
evaluator = sime.run_sagemaker_evaluation(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    y_test_pred,
    y_val_pred
)

In [ ]:
evaluator.y_validation

In [ ]:
evaluator.y_validation_pred_proba

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(evaluator.y_validation, evaluator.y_validation_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Validation Set')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
evaluator.summary_report()
evaluator.plot_wallet_evaluation()


In [ ]:
def load_sagemaker_predictions(
    data_type: str,
    sage_wallets_config: dict,
    sage_wallets_modeling_config: dict,
    date_suffix: str
) -> tuple[pd.Series, pd.Series]:
    """
    Load SageMaker predictions and corresponding actuals for a given data type.

    Params:
    - data_type (str): Either 'test' or 'val'
    - sage_wallets_config (dict): Configuration for training data paths
    - sage_wallets_modeling_config (dict): Configuration for model parameters
    - date_suffix (str): Date suffix for file naming

    Returns:
    - tuple: (predictions_series, actuals_series) with aligned indices
    """
    # Load predictions
    pred_path = Path(sage_wallets_modeling_config['metaparams']['endpoint_preds_dir']) / \
                f"endpoint_y_pred_{data_type}_{sage_wallets_config['training_data']['local_directory']}_{date_suffix}.csv"
    pred_df = pd.read_csv(pred_path)

    if 'score' not in pred_df.columns:
        raise ValueError(f"SageMaker predictions are missing the 'score' column. "
                        f"Available columns: {pred_df.columns}")
    pred_series = pred_df['score']

    # Load actuals
    training_data_path = (
        Path(f"../s3_uploads") / "wallet_training_data_queue" /
        f"{sage_wallets_config['training_data']['local_directory']}"
    )
    actuals_path = training_data_path / f"y_{data_type}_{date_suffix}.parquet"
    actuals_df = pd.read_parquet(actuals_path)

    if len(actuals_df.columns) > 1:
        raise ValueError(f"Found unexpected columns in y_{data_type}_df. "
                        f"Expected 1 column, found {actuals_df.columns}.")
    actuals_series = actuals_df.iloc[:, 0]

    # Validate lengths and align indices
    if len(pred_series) != len(actuals_series):
        raise ValueError(f"Length of y_{data_type}_pred ({len(pred_series)}) does "
                        f"not match length of y_{data_type}_true ({len(actuals_series)}).")

    pred_series.index = actuals_series.index

    return pred_series, actuals_series

In [ ]:
# Load predictions and actuals
y_test_pred_series, y_test_true_series = load_sagemaker_predictions(
    'test', sage_wallets_config, sage_wallets_modeling_config, date_suffix
)
y_val_pred_series, y_val_true_series = load_sagemaker_predictions(
    'val', sage_wallets_config, sage_wallets_modeling_config, date_suffix
)

# Load remaining training data
training_data_path = (
    Path(f"../s3_uploads") / "wallet_training_data_queue" /
    f"{sage_wallets_config['training_data']['local_directory']}"
)
X_train = pd.read_parquet(training_data_path / f"x_train_{date_suffix}.parquet")
y_train = pd.read_parquet(training_data_path / f"y_train_{date_suffix}.parquet")
X_test = pd.read_parquet(training_data_path / f"x_test_{date_suffix}.parquet")
X_val = pd.read_parquet(training_data_path / f"x_val_{date_suffix}.parquet")

# Identify target variable and model type
target_variable = y_val_true_series.name or y_train.columns[0]
objective = sage_wallets_modeling_config['training']['hyperparameters']['objective']
model_type = 'regression' if objective[:3] == 'reg' else 'unknown'

In [ ]:
# Create model_id and modeling_config
model_id = f"sagemaker_{sage_wallets_config['training_data']['local_directory']}_{date_suffix}"

modeling_config = {
    'target_variable': target_variable,
    'model_type': model_type,
    'returns_winsorization': 0.005,  # Default for winsorizing returns
    'training_data': {
        'modeling_period_duration': 30  # Default performance window
    },
    'sagemaker_metadata': {
        'objective': objective,
        'local_directory': sage_wallets_config['training_data']['local_directory'],
        'date_suffix': date_suffix
    }
}

# Create minimal wallet_model_results for SageMaker evaluation
wallet_model_results = {
    'model_id': model_id,
    'modeling_config': modeling_config,
    'model_type': model_type,

    # Training data
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test_true_series,
    'y_pred': y_test_pred_series,
    'training_cohort_pred': None,
    'training_cohort_actuals': None,

    # Validation data
    'X_validation': X_val,
    'y_validation': y_val_true_series,
    'y_validation_pred': y_val_pred_series,
    'validation_target_vars_df': None,

    # Fixed mock pipeline
    'pipeline': type('MockPipeline', (), {
        'named_steps': {'estimator': type('MockModel', (), {
            'get_params': lambda self: {'objective': objective}  # Accept self argument
        })()},
        '__getitem__': lambda self, key: type('MockTransformer', (), {
            'transform': lambda self, X: X  # Accept self argument
        })()
    })()
}

# Create evaluator
wallet_evaluator = wime.RegressorEvaluator(wallet_model_results)

# Run basic evaluation
wallet_evaluator.summary_report()
wallet_evaluator.plot_wallet_evaluation()

In [ ]:
target_variable